In [1]:
import pickle 
import numpy as np
import pandas as pd
from classes_colors import *
import matplotlib.pyplot as plt

In [2]:
with open("sensor_labels.pkl", "rb") as f:
    sensor_labels = pickle.load(f)

with open("sensor_data_w_missing_vals.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [16]:
def build_comparison_df(sensor_data, day, mat, sensor):
    data_lengths = []
    for heater_idx in sensor_data[f"day{day}"][f"mat{mat}"][sensor]:
        heater_data_len = sensor_data[f"day{day}"][f"mat{mat}"][sensor][heater_idx].shape[0]
        data_lengths.append(heater_data_len)
    
    min_len = min(data_lengths)
    heater_dict = {}
    for heater_idx in sensor_data[f"day{day}"][f"mat{mat}"][sensor]:
        heater_dates = sensor_data[f"day{day}"][f"mat{mat}"][sensor][heater_idx]["Date"].to_numpy()
        heater_data_len = len(heater_dates)
        if heater_data_len > min_len:
            heater_dates = heater_dates[:min_len]
        heater_dict[heater_idx] = heater_dates
    
    for heater_idx in range(1, 10):
        diff_seconds = (heater_dict[0] - heater_dict[heater_idx]).astype("timedelta64[s]").astype(np.int32)
        heater_dict[f"d 0-{heater_idx}"] = diff_seconds
    
    df = pd.DataFrame(heater_dict)
    return df 

In [96]:
def impute_data_mean(df, after):
    df_copy = df.copy()
    line = df_copy.iloc[after:after+2].mean()

    line["Sensor Index"] = int(line["Sensor Index"])
    line["Sensor ID"] = int(line["Sensor ID"])
    line["Time Since PowerOn"] = int(line["Time Since PowerOn"])
    line["Real time clock"] = int(line["Real time clock"])
    line["Heater Profile Step Index"] = int(line["Heater Profile Step Index"])
    line["Scanning Mode Enabled"] = int(line["Scanning Mode Enabled"])
    line["Scanning Cycle Index"] = int(line["Scanning Cycle Index"])
    line["Label Tag"] = int(line["Label Tag"])
    line["Error Code"] = int(line["Error Code"])
    line["Date"] = line["Date"].floor("S")

    index = df_copy.iloc[after:after+2].index
    index = int((index[0] + index[1]) / 2)
    df_copy.loc[index] = line
    df_copy.sort_index(inplace=True)
    return df_copy

In [115]:
day = 1
matrix = 1
for i in range(8):
    df = build_comparison_df(sensor_data, day, matrix, i)
    df.to_excel(f"d{day}_m{matrix}_s{i}_diff.xlsx")

In [ ]:
sensor = 3
heater = 2
after = 13
df_copy = impute_data_mean(sensor_data[f"day{day}"][f"mat{matrix}"][sensor][heater], first)
sensor_data[f"day{1}"][f"mat{1}"][sensor][heater] = df_copy.copy()

In [114]:
with open("sensor_data_imputed.pkl", "wb") as f:
    pickle.dump(sensor_data, f)